In [1]:
# this is an experiment notebook to test whether reloading works.
# I was having a lot of issue when reloading the model. 
# but eventually got it working as expected
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.keras import backend as K
from keras.models import Model, Input, model_from_yaml, load_model
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
from keras.utils import plot_model
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
n_tags = 11
max_len = 496
batch_size = 8
def ElmoEmbedding(x):
    return elmo_model(inputs={"tokens": tf.squeeze(tf.cast(x,tf.string)),"sequence_len": tf.constant(batch_size*[max_len])
                     },
                      signature="tokens",
                      as_dict=True)["elmo"]

Using TensorFlow backend.


In [2]:
reloadModel = load_model(r'./Models/elmoModel', custom_objects={'ElmoEmbedding': ElmoEmbedding, 'elmo_model':elmo_model, 'tf':tf, 'batch_size': batch_size, 'max_len': max_len, 'n_tags': n_tags}, compile = True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
reloadModel.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 496)          0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 496, 1024)    0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 496, 1024)    6295552     lambda_1[0][0]                   
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 496, 1024)    6295552     bidirectional_1[0][0]            
____________________________________________________________________________________________

In [4]:
import json
with open('tags2index.json') as f:
    tagDecoder = json.load(f)
tagDecoder = {value:key for key, value in tagDecoder.items()}

In [5]:
tagDecoder

{1: 'O',
 2: 'B-TIME',
 3: 'I-GST',
 4: 'B-MERCHANT',
 5: 'I-AMOUNT',
 6: 'B-AMOUNT',
 7: 'I-TIME',
 8: 'B-DATE',
 9: 'I-MERCHANT',
 10: 'I-DATE',
 11: 'B-GST',
 0: 'PADtoken'}

In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")
inputInvoice = 'CoronwealthBank, MONGAB PTY LTD, PORT MELBOU VIC, MERCHANT COPY, CARD NUMBER 4999-2669(c, EXPIRY DATE, 10/21, VISA CARD, VISA, CREDIT, PURCHASE, $41.92, TOTAL, AUD $41.92, APPROVED, 00, 14 JUL 2018, 11:11, TERMINAL ID, 17605400, REFERENCE, 061083, AUTH NUMBER, 163837, AID, A000y000031010, ATC 0024, CSN 01, TVR TC 22FC88EE5B8 22FC88EE5BSAF189, THANK YOU' #raw_input().lower()

doc = nlp(inputInvoice.lower())
tokens = [token.text for token in doc]
token_len = len(tokens)
new_token = []
newInput = []
if len(tokens) <= 0:
    raise Exception('tokenisation failed.')
elif len(tokens) > max_len:
    print('to-do')
elif len(tokens) > 0 and len(tokens) <= max_len:
    for i in range(max_len):
        try:
            new_token.append(tokens[i])
        except:
            new_token.append('PADtoken')
    newInput= [new_token]*batch_size
else:
    raise Exception('Unkown value error, at padding process.')

In [7]:
len(newInput)

8

In [8]:
prediction = ''
with tf.Session(config = config) as sess:
    K.set_session(sess)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    prediction = reloadModel.predict(np.array(newInput), verbose=1, batch_size = 8)

8/8 [==============================] - 9s 1s/step


In [12]:
idx2tag = tagDecoder
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PADtoken", "O"))
        out.append(out_i)
    return out
test = pred2label(prediction)

In [13]:
prediction[0]

array([[0.05379399, 0.05573447, 0.04600727, ..., 0.10139935, 0.1218928 ,
        0.08430117],
       [0.05008829, 0.04885332, 0.04155744, ..., 0.10763133, 0.11444972,
        0.08476812],
       [0.0426067 , 0.0432939 , 0.03890711, ..., 0.12128436, 0.12809029,
        0.09515259],
       ...,
       [0.04806637, 0.06218767, 0.07143959, ..., 0.08129038, 0.15698133,
        0.07169455],
       [0.0487371 , 0.06388604, 0.0747645 , ..., 0.08633029, 0.14179875,
        0.07402553],
       [0.05167875, 0.06804562, 0.07859909, ..., 0.0916351 , 0.12159611,
        0.07266004]], dtype=float32)

In [22]:
test[0]
np.argmax()

3

In [24]:
test[0]

['I-GST',
 'I-GST',
 'I-DATE',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-MERCHANT',
 'I-GST',
 'I-MERCHANT',
 'I-GST',
 'I-MERCHANT',
 'I-MERCHANT',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'B-AMOUNT',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-MERCHANT',
 'I-MERCHANT',
 'I-AMOUNT',
 'I-MERCHANT',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'B-AMOUNT',
 'B-AMOUNT',
 'B-AMOUNT',
 'B-AMOUNT',
 'B-AMOUNT',
 'B-AMOUNT',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'B-AMOUNT',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST',
 'I-GST

In [25]:
tokens

'CoronwealthBank'